In [331]:
import pandas as pd
import numpy as np
from string import ascii_lowercase
import itertools
import string
from collections import defaultdict

pd.set_option('display.max_columns', 50)

### Board

In [332]:
connect_n = 4
board_height = 6
board_width = 7 # max 26

In [333]:
columns = [column for column in ascii_lowercase[:board_width]]
rows = [row for row in range(board_height)]

In [334]:
X_cols = []
for column in ascii_lowercase[:board_width]:
    for row in range(board_height):
        X_cols.append(column + str(row))

In [335]:
def new_game(X_cols):
    return pd.Series(0, X_cols)

In [336]:
def empty_positions(rows, columns):
    return {col:[row for row in rows] for col in columns}

### Winning positions

In [337]:
np.random.seed(seed = 1)
X_rand = pd.DataFrame(np.random.randint(3, size=(1000, len(X_cols)), ), columns=X_cols) - 1
win_types = ['up', 'across', 'diag /', 'diag \\']
win_type = 'diag \\'

In [338]:
def rows_and_columns_checked(win_type, all_rows, all_columns):
    win_type_positions = {}
    limit_top_rows = all_rows[:-connect_n + 1]
    limit_bottom_rows = all_rows[connect_n - 1:]
    limit_right_columns =  all_columns[:-connect_n + 1]
    
    # create dictionary of win types and a tuple of the rows and columns checked for each win type
    for win_type in win_types:
        if win_type == 'up':
            rows_checked, columns_checked = limit_top_rows, all_columns
        elif win_type == 'across':
            rows_checked, columns_checked = all_rows, limit_right_columns
        elif win_type == 'diag /':
            rows_checked, columns_checked = limit_top_rows, limit_right_columns
        elif win_type == 'diag \\':
            rows_checked, columns_checked = limit_bottom_rows, limit_right_columns
        else:
            raise ValueError('not a valid win type')
        win_type_positions[win_type] = rows_checked, columns_checked

    # create dictionary of individual row/column positions and a list of their possible win types 
    positions_win_types_to_check = defaultdict(list)
    for win_type, rows_and_columns_to_check in win_type_positions.iteritems():
        for positions in itertools.product(rows_and_columns_to_check[0],rows_and_columns_to_check[1]):
            row = positions[0]
            column = positions[1]
            position = column + str(row)
            positions_win_types_to_check[position, (row, column)].append(win_type)
    return positions_win_types_to_check

In [339]:
def win_type_angle(win_type, row_checked, column_checked):

    # list row and column positions
    if win_type == 'up':
        angle_column_positions = list(column_checked) * connect_n
        angle_row_positions = list(map(str, range(row_checked, row_checked + connect_n)))
    elif win_type == 'across':
        angle_column_positions = list(map(chr, range(ord(column_checked), ord(column_checked) + connect_n)))
        angle_row_positions = list(str(row_checked)) * connect_n
    elif win_type == 'diag /':
        angle_column_positions = list(map(chr, range(ord(column_checked), ord(column_checked) + connect_n)))
        angle_row_positions = list(map(str, range(row_checked, row_checked + connect_n)))
    elif win_type == 'diag \\':
        angle_column_positions = list(map(chr, range(ord(column_checked), ord(column_checked) + connect_n)))
        angle_row_positions = list(map(str, range(row_checked, row_checked - connect_n, -1)))
    else:
        raise ValueError('not a valid win type')

    # combine postitions
    return [column + row for column, row in zip(angle_column_positions, angle_row_positions)]

In [340]:
def get_winning_positions(positions_win_types_to_check):
    winning_positions = defaultdict(list)
    for position, position_win_types in positions_win_types_to_check.iteritems():
        row = position[1][0]
        column = position[1][1]
        for win_type in position_win_types:
            additional_positions_to_check = win_type_angle(win_type, row, column)
            winning_positions[position[0]].append((win_type, additional_positions_to_check))
    return winning_positions

In [341]:
positions_win_types_to_check = rows_and_columns_checked(win_type, rows, columns)

In [342]:
winning_positions = get_winning_positions(positions_win_types_to_check)

### Check for win

In [343]:
game = new_game(X_cols)

In [344]:
remaining_empty_positions = empty_positions(rows, columns)

In [346]:
player = -1

In [347]:
def available_moves(player, remaining_empty_positions):
    return [col + str(rows[0]) for col, rows in remaining_empty_positions.iteritems()]

In [348]:
def remove_empty_position(col, remaining_empty_positions):
    remaining_empty_positions[col].pop(0)
    return remaining_empty_positions

In [349]:
def check_positions(game):
    return game[game != 0].index

In [374]:
def obvious_win(game, remaining_empty_positions, winning_positions, player):
    # has_best_move  BOOLEAN
    # move_to_make   STRING board_position OR NULL
    for board_position in check_positions(game):
        for win_type_angle in winning_positions[board_position]:
            game_positions_of_win_type_angle = game[win_type_angle[1]]
            win_type_angle_status = game[win_type_angle[1]].value_counts()
            if len(win_type_angle_status) == 2 and 0 in win_type_angle_status.index and player in win_type_angle_status.index:
                if win_type_angle_status.loc[0] == 1 and win_type_angle_status.loc[player] == 3:
                    winning_move = game_positions_of_win_type_angle[game_positions_of_win_type_angle == 0].index[0]
                    if winning_move in available_moves(player, remaining_empty_positions):
                        return True, winning_move
    return False, None

In [351]:
game = X_rand.iloc[6]

In [364]:
remaining_empty_positions = remove_empty_position('d', remaining_empty_positions)

In [371]:
obvious_win(game, remaining_empty_positions, winning_positions, player)

(False, None)

In [372]:
positions_to_check(game)

Index([u'a0', u'a1', u'a2', u'a3', u'a4', u'a5', u'b1', u'b2', u'b3', u'b4',
       u'b5', u'c2', u'c3', u'c4', u'd0', u'd2', u'd4', u'd5', u'e0', u'e1',
       u'e2', u'e3', u'e5', u'f0', u'f1', u'f2', u'f4', u'f5', u'g0', u'g1',
       u'g2', u'g4', u'g5'],
      dtype='object')

In [50]:
df_new_game

,a0,a1,a2,a3,a4,a5,b0,b1,b2,b3,b4,b5,c0,c1,c2,c3,c4,c5,d0,d1,d2,d3,d4,d5,e0,e1,e2,e3,e4,e5,f0,f1,f2,f3,f4,f5,g0,g1,g2,g3,g4,g5
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
def make_move(move_to_make, player):
    # new_board_positions  DICT all board positions
    return new_board_positions

In [ ]:
def evaluate_moves(player):
    # move_to_make   STRING board_position OR NULL
    return move_to_make

In [ ]:
def check_for_win(new_board_positions, player):
    # player    INT 1 or -1 
    # position  STRING board_position
    # win_type  STRING 'up, across, etc.
    
    return (player, position, win_type)

In [ ]:
def make_move(player):
    # check for obvious wins
    has_best_move, move_to_make = obvious_win(player)
    if has_best_move = True:
        new_board_positions = make_move(move_to_make, player)
    # evaluate possible next moves
    else:
        move_to_make = evaluate_moves(player)
        new_board_positions = make_move(move_to_make, player)
    # check for win
    return check_for_win(new_board_positions, player)

In [27]:
np.random.seed(seed = 1)
X_rand = pd.DataFrame(np.random.randint(3, size=(1000, len(X_cols)), ), columns=X_cols) - 1

In [28]:
winning_info = {}
for idx, board_positions in X_rand[:10].iterrows():
    winning_info[idx] = 0
    for position, token in board_positions[board_positions.isin([-1, 1])].iteritems():
        for win_type, positions in winning_positions[position]:
            if board_positions[positions].map(lambda x: x == token).all():
                winning_info[idx] = token
                print str(idx) + ' {} | wins in {} position | with a {} win type'.format(token, position, win_type)

1 -1 | wins in b5 position | with a diag \ win type
1 -1 | wins in c4 position | with a diag \ win type
1 -1 | wins in d3 position | with a diag \ win type
2 -1 | wins in d3 position | with a across win type
2 -1 | wins in d5 position | with a diag \ win type
2 -1 | wins in g2 position | with a up win type
3 1 | wins in a5 position | with a diag \ win type
3 1 | wins in a5 position | with a across win type
3 1 | wins in b4 position | with a diag \ win type
5 -1 | wins in a0 position | with a across win type
5 1 | wins in e0 position | with a up win type
5 -1 | wins in f2 position | with a up win type
6 -1 | wins in b1 position | with a up win type
8 1 | wins in a5 position | with a diag \ win type
9 -1 | wins in c0 position | with a across win type
9 1 | wins in f1 position | with a up win type


In [13]:
initial_example = X_rand.loc[6:8]

In [35]:
initial_example.iloc[:2, :3].to_dict()

{'a0': {6: 1, 7: -1}, 'a1': {6: 1, 7: -1}, 'a2': {6: 1, 7: -1}}

In [15]:
dict((k, winning_info[k]) for k in (6, 7, 8))

{6: -1, 7: 0, 8: 1}

In [16]:
winning_info[]

SyntaxError: invalid syntax (<ipython-input-16-c2673684f405>, line 1)

a0    1
a1    1
a2    1
a3   -1
a4    1
a5    1
b0    0
b1   -1
b2   -1
b3   -1
b4   -1
b5    1
c0    0
c1    0
c2   -1
c3    1
c4    1
c5    0
d0    1
d1    0
d2    1
d3    0
d4    1
d5   -1
e0   -1
e1    1
e2    1
e3   -1
e4    0
e5    1
f0   -1
f1    1
f2   -1
f3    0
f4   -1
f5   -1
g0    1
g1   -1
g2    1
g3    0
g4    1
g5   -1
Name: 6, dtype: int64